In [2]:
#importing libraries in the notebook
import pandas as pd
import numpy as np
from urllib import parse
from sqlalchemy import create_engine
import pandas as pd

In [1]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sqlalchemy

     |████████████████████████████████| 1.6 MB 23.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# adding admin levels to Brazil master

In [3]:
d_r = pd.read_csv('BR_school_geolocation_master.csv',low_memory=False,encoding='utf-8')

In [4]:
d_r.head()

,uuid,country,source,giga_school_id,school_id,name,lat,lon,hex8,dq_score,...,source_timestamp,master_timestamp,school_archive,district_code,student_count,computer_availability,school_type,school_region,connectivity,internet_speed_Mbps
0,82679ff0-f358-33d0-b3ce-4fc4c2740b7e,Brazil,government,G_BRA_000001,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,-8.7585,-63.8540,888a05915dfffff,0.94,...,2021-04,32:36.9,No,110020505,174,Yes,Estadual,Urban,Yes,5.0
1,115d27ba-82a1-3744-a3fb-90f216f54f6f,Brazil,government,G_BRA_000002,11000040,EMEIEF PEQUENOS TALENTOS,-8.7937,-63.8839,888a059319fffff,1.00,...,2021-04,32:36.9,No,110020505,205,No,Municipal,Urban,Yes,5.0
2,3d7fb661-61a4-33b3-bc3a-33455b75f7d9,Brazil,government,G_BRA_000003,11000201,EMEF PROF HERBERT DE ALENCAR,-8.7640,-63.8783,888a059061fffff,0.94,...,2021-04,32:36.9,No,110020505,407,Yes,Municipal,Urban,Yes,5.0
3,9507b9e6-4ea6-3cfc-b0c5-4c0b9f64c711,Brazil,government,G_BRA_000004,11000260,COLEGIO TIRADENTES DA POLICIA MILITAR DO ESTAD...,-8.7383,-63.8791,888a059021fffff,1.00,...,2021-04,32:36.9,No,110020505,1457,Yes,Estadual,Urban,Yes,5.0
4,b5ec97ef-a418-35da-b161-3971c87ac36d,Brazil,government,G_BRA_000005,11000309,ESCOLA MUNICIPAL DE EDUCACAO INFANTIL MARISE C...,-8.7508,-63.9087,888a05900bfffff,1.00,...,2021-04,32:36.9,No,110020505,499,No,Municipal,Urban,No,0.0


In [5]:
d_r.shape

(175704, 21)

In [10]:
d_r.rename(columns={'lon': 'longitude', 'lat': 'latitude'}, inplace=True)

In [6]:
import geopandas as gpd
import shapely.speedups
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import rtree

In [7]:
admin3_gdf = gpd.read_file("gadm36_BRA.gpkg", layer='gadm36_BRA_3')
admin3_gdf.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,GID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
0,BRA,Brazil,BRA.1_1,Acre,None,BRA.1.1_1,Acrelândia,None,BRA.1.1.1_1,Acrelândia,None,None,Distrito,District,None,None,"MULTIPOLYGON (((-67.16084 -9.98260, -67.15184 ..."
1,BRA,Brazil,BRA.1_1,Acre,None,BRA.1.2_1,Assis Brazil,None,BRA.1.2.1_1,Assis Brazil,None,None,Distrito,District,None,None,"MULTIPOLYGON (((-69.56518 -10.95550, -69.56603..."
2,BRA,Brazil,BRA.1_1,Acre,None,BRA.1.3_1,Brasiléia,None,BRA.1.3.1_1,Brazileia,None,None,Distrito,District,None,None,"MULTIPOLYGON (((-69.42356 -10.92717, -69.41938..."
3,BRA,Brazil,BRA.1_1,Acre,None,BRA.1.4_1,Bujari,None,BRA.1.4.1_1,Bujari,None,None,Distrito,District,None,None,"MULTIPOLYGON (((-67.94034 -9.33722, -67.94411 ..."
4,BRA,Brazil,BRA.1_1,Acre,None,BRA.1.5_1,Capixaba,None,BRA.1.5.1_1,Capixaba,None,None,Distrito,District,None,None,"MULTIPOLYGON (((-68.00378 -10.65377, -68.00352..."


In [13]:
d_r['geometry'] = d_r.apply(lambda z: Point(z.longitude, z.latitude), axis=1)
PointsGeodataframe = gpd.GeoDataFrame(d_r)
v1 = len(PointsGeodataframe)
print(v1)
PointsGeodataframe['valid'] = PointsGeodataframe.is_valid
display(PointsGeodataframe.loc[PointsGeodataframe['valid'] == False])
missing_location = PointsGeodataframe[PointsGeodataframe['valid'] == False]
PointsGeodataframe = PointsGeodataframe[PointsGeodataframe['valid'] == True]
print(PointsGeodataframe.columns)
v2 = len(PointsGeodataframe)
print(v2)

175704


,uuid,country,source,giga_school_id,school_id,name,latitude,longitude,hex8,dq_score,...,school_archive,district_code,student_count,computer_availability,school_type,school_region,connectivity,internet_speed_Mbps,geometry,valid


Index(['uuid', 'country', 'source', 'giga_school_id', 'school_id', 'name',
       'latitude', 'longitude', 'hex8', 'dq_score', 'duplicate',
       'source_timestamp', 'master_timestamp', 'school_archive',
       'district_code', 'student_count', 'computer_availability',
       'school_type', 'school_region', 'connectivity', 'internet_speed_Mbps',
       'geometry', 'valid'],
      dtype='object')
175704


In [14]:
dfb = gpd.sjoin(PointsGeodataframe, admin3_gdf, op='within') 
print(dfb.columns)
display(dfb)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-14-0a9a1cabc849>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  dfb = gpd.sjoin(PointsGeodataframe, admin3_gdf, op='within')


Index(['uuid', 'country', 'source', 'giga_school_id', 'school_id', 'name',
       'latitude', 'longitude', 'hex8', 'dq_score', 'duplicate',
       'source_timestamp', 'master_timestamp', 'school_archive',
       'district_code', 'student_count', 'computer_availability',
       'school_type', 'school_region', 'connectivity', 'internet_speed_Mbps',
       'geometry', 'valid', 'index_right', 'GID_0', 'NAME_0', 'GID_1',
       'NAME_1', 'NL_NAME_1', 'GID_2', 'NAME_2', 'NL_NAME_2', 'GID_3',
       'NAME_3', 'VARNAME_3', 'NL_NAME_3', 'TYPE_3', 'ENGTYPE_3', 'CC_3',
       'HASC_3'],
      dtype='object')


,uuid,country,source,giga_school_id,school_id,name,latitude,longitude,hex8,dq_score,...,NAME_2,NL_NAME_2,GID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3
0,82679ff0-f358-33d0-b3ce-4fc4c2740b7e,Brazil,government,G_BRA_000001,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,-8.7585,-63.8540,888a05915dfffff,0.94,...,Porto Velho,None,BRA.22.36.16_1,Zona 04,None,None,Distrito,District,None,None
8,b42536af-edfb-3871-b0a9-387ff8405692,Brazil,government,G_BRA_000009,11000384,EEEFM 4 DE JANEIRO,-8.7414,-63.8643,888a059119fffff,1.00,...,Porto Velho,None,BRA.22.36.16_1,Zona 04,None,None,Distrito,District,None,None
21,52da1f40-b1de-34aa-9fe3-7903a579e40a,Brazil,government,G_BRA_000022,11000660,EMEIEF BOM PRINCIPIO,-8.7622,-63.8372,888a059145fffff,1.00,...,Porto Velho,None,BRA.22.36.16_1,Zona 04,None,None,Distrito,District,None,None
26,8dcae9b2-fe41-3407-a26a-19d53f33560f,Brazil,government,G_BRA_000027,11000856,EEEFM PROFESSOR DANIEL NERI DA SILVA,-8.7664,-63.8363,888a059145fffff,1.00,...,Porto Velho,None,BRA.22.36.16_1,Zona 04,None,None,Distrito,District,None,None
33,eb011aeb-fc94-3339-98cc-6303b66d9cb4,Brazil,government,G_BRA_000034,11001003,EMEF ENG,-8.7768,-63.8541,888a059335fffff,1.00,...,Porto Velho,None,BRA.22.36.16_1,Zona 04,None,None,Distrito,District,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172547,c3412cbf-f6f2-39b0-b4a0-4a7112f2fad6,Brazil,government,G_BRA_172548,31200514,EM JOAO DIAS ROSA,-18.5920,-47.6665,88a8f03845fffff,0.94,...,Estrela do Sul,None,BRA.13.280.1_1,Chapada de Minas,None,None,Distrito,District,None,None
173027,8ae6bb3c-b62f-3c22-a34a-92e0831a25fc,Brazil,government,G_BRA_173028,31119695,EM PIO CANDIDO DE OLIVEIRA,-18.5886,-45.7280,88a88d2069fffff,0.94,...,Tiros,None,BRA.13.811.1_1,Canastrao,None,None,Distrito,District,None,None
173806,d7c4c8fa-cda5-3dc7-942c-394786eab480,Brazil,government,G_BRA_173807,26100215,ESCOLA MUNICIPAL MARCELINA MARIA DE AMORIM,-8.6205,-35.4289,888181db61fffff,1.00,...,Gameleira,None,BRA.17.65.1_1,Cuiambuca,None,None,Distrito,District,None,None
173895,fe68cb94-8db8-35a3-9f9e-4c3f36d1ac78,Brazil,government,G_BRA_173896,25030124,EEEF OLIVINA CARNEIRO DA CUNHA,-7.5082,-38.3492,8880365915fffff,1.00,...,Ibiara,None,BRA.15.86.1_1,Cachoeirinha,None,None,Distrito,District,None,None


In [15]:
Bdf = dfb.drop(['index_right', 'GID_0', 'GID_1',
       'NL_NAME_1', 'GID_2', 'NL_NAME_2', 'GID_3',
        'TYPE_3', 'ENGTYPE_3', 'CC_3', 'HASC_3', 'VARNAME_3', 'NL_NAME_3'], axis = 1)
Bdf.reset_index(inplace = True)
print(Bdf.columns)

Index(['index', 'uuid', 'country', 'source', 'giga_school_id', 'school_id',
       'name', 'latitude', 'longitude', 'hex8', 'dq_score', 'duplicate',
       'source_timestamp', 'master_timestamp', 'school_archive',
       'district_code', 'student_count', 'computer_availability',
       'school_type', 'school_region', 'connectivity', 'internet_speed_Mbps',
       'geometry', 'valid', 'NAME_0', 'NAME_1', 'NAME_2', 'NAME_3'],
      dtype='object')


In [16]:
Bdf.head()

,index,uuid,country,source,giga_school_id,school_id,name,latitude,longitude,hex8,...,school_type,school_region,connectivity,internet_speed_Mbps,geometry,valid,NAME_0,NAME_1,NAME_2,NAME_3
0,0,82679ff0-f358-33d0-b3ce-4fc4c2740b7e,Brazil,government,G_BRA_000001,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,-8.7585,-63.8540,888a05915dfffff,...,Estadual,Urban,Yes,5.0,POINT (-63.85400 -8.75850),True,Brazil,Rondônia,Porto Velho,Zona 04
1,8,b42536af-edfb-3871-b0a9-387ff8405692,Brazil,government,G_BRA_000009,11000384,EEEFM 4 DE JANEIRO,-8.7414,-63.8643,888a059119fffff,...,Estadual,Urban,Yes,5.0,POINT (-63.86430 -8.74140),True,Brazil,Rondônia,Porto Velho,Zona 04
2,21,52da1f40-b1de-34aa-9fe3-7903a579e40a,Brazil,government,G_BRA_000022,11000660,EMEIEF BOM PRINCIPIO,-8.7622,-63.8372,888a059145fffff,...,Municipal,Urban,Yes,5.0,POINT (-63.83720 -8.76220),True,Brazil,Rondônia,Porto Velho,Zona 04
3,26,8dcae9b2-fe41-3407-a26a-19d53f33560f,Brazil,government,G_BRA_000027,11000856,EEEFM PROFESSOR DANIEL NERI DA SILVA,-8.7664,-63.8363,888a059145fffff,...,Estadual,Urban,Yes,5.0,POINT (-63.83630 -8.76640),True,Brazil,Rondônia,Porto Velho,Zona 04
4,33,eb011aeb-fc94-3339-98cc-6303b66d9cb4,Brazil,government,G_BRA_000034,11001003,EMEF ENG,-8.7768,-63.8541,888a059335fffff,...,Municipal,Urban,Yes,5.0,POINT (-63.85410 -8.77680),True,Brazil,Rondônia,Porto Velho,Zona 04


In [17]:
Bdf.shape

(175704, 28)

In [18]:
Bdf.to_csv("Bdf.csv",encoding='utf-8')

In [8]:
rw_m = pd.read_csv('./giga/master/school_location/rwanda_school_geolocation_master.csv',low_memory=False,encoding='utf-8')

In [9]:
rw_m.head()

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,dq_score,source_timestamp,master_timestamp,school_archive,electricity_availability,internet_availability,number_of_students,number_of_teachers,computers,connectivity_type
0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,886ade34e7fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,28,1,260.0,3G
1,a58be9dc-59c4-4cf9-96f5-e5b8fa84ba82,Rwanda,government,G_RWA_00005,331220.0,GS MWENDO,Secondary,-1.730639,30.212722,886ad88707fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,62,3,272.0,3G
2,9fe38ae7-7d51-4646-841b-1b6daa31c987,Rwanda,government,G_RWA_00006,411305.0,GS NYAGISOZI,Secondary,-2.295028,29.645556,886ade2481fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,64,4,218.0,3G
3,8b5c3930-c3ea-44d8-a510-086f8c6889aa,Rwanda,government,G_RWA_00009,510619.0,GS RUGANGO,Secondary,-2.371833,30.599167,886ad821d7fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,80,2,252.0,3G
4,2288686a-e13e-423a-b0bb-b7d08452e9a8,Rwanda,government,G_RWA_00012,130610.0,GS GIHINGA,Secondary,-1.926806,30.591806,886ad8a095fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,98,3,259.0,3G


In [10]:
rw_1436 = pd.read_csv('./data/1436_rw.csv',low_memory=False,encoding='utf-8')

In [11]:
rw_1436.head()

,School Name_Mineduc,school_code,Province_mineduc,District_mineduc,Longitude,Latitude
0,COLLEGE ST FRANCOIX XAVIER APEBU NYAMATA TVET ...,571002,East,Bugesera,30.097028,-2.154389
1,EP Kindama,571302,East,Bugesera,30.086833,-2.322472
2,EP MBYO SCHOOL,570510,East,Bugesera,30.156750,-2.229389
3,EP Nemba,571404,East,Bugesera,30.217861,-2.340694
4,ES JURU,570201,EAST,BUGESERA,30.188944,-2.101500


In [12]:
rw_m.shape

(4093, 20)

In [13]:
rw_m.dtypes

uuid                         object
country                      object
source                       object
giga_school_id               object
source_school_id            float64
school_name                  object
education_level              object
latitude                    float64
longitude                   float64
hex8                         object
dq_score                    float64
source_timestamp              int64
master_timestamp             object
school_archive               object
electricity_availability     object
internet_availability        object
number_of_students            int64
number_of_teachers            int64
computers                   float64
connectivity_type            object
dtype: object

In [14]:
rw_1436.dtypes

School Name_Mineduc     object
school_code              int64
Province_mineduc        object
District_mineduc        object
Longitude              float64
Latitude               float64
dtype: object

In [18]:
df2 = rw_m.loc[~((rw_m.source_school_id.isin(rw_1436['school_code'])))]


In [19]:
df2.shape

(3240, 20)

In [20]:
rw_1436.shape

(873, 6)

In [21]:
df2.head()

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,dq_score,source_timestamp,master_timestamp,school_archive,electricity_availability,internet_availability,number_of_students,number_of_teachers,computers,connectivity_type
0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,886ade34e7fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,28,1,260.0,3G
1,a58be9dc-59c4-4cf9-96f5-e5b8fa84ba82,Rwanda,government,G_RWA_00005,331220.0,GS MWENDO,Secondary,-1.730639,30.212722,886ad88707fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,62,3,272.0,3G
2,9fe38ae7-7d51-4646-841b-1b6daa31c987,Rwanda,government,G_RWA_00006,411305.0,GS NYAGISOZI,Secondary,-2.295028,29.645556,886ade2481fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,64,4,218.0,3G
3,8b5c3930-c3ea-44d8-a510-086f8c6889aa,Rwanda,government,G_RWA_00009,510619.0,GS RUGANGO,Secondary,-2.371833,30.599167,886ad821d7fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,80,2,252.0,3G
4,2288686a-e13e-423a-b0bb-b7d08452e9a8,Rwanda,government,G_RWA_00012,130610.0,GS GIHINGA,Secondary,-1.926806,30.591806,886ad8a095fffff,0.67,2020,00:52.6,No,ON GRID,USB_MODEM,98,3,259.0,3G


In [22]:
admin3_gdf = gpd.read_file("./data/gadm36_RWA.gpkg", layer='gadm36_RWA_3')
admin3_gdf.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,GID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
0,RWA,Rwanda,RWA.1_1,Amajyaruguru,None,RWA.1.1_1,Burera,None,RWA.1.1.1_1,Bungwe,None,None,Sector,Sector,4401,None,"MULTIPOLYGON (((29.96161 -1.52627, 29.96132 -1..."
1,RWA,Rwanda,RWA.1_1,Amajyaruguru,None,RWA.1.1_1,Burera,None,RWA.1.1.2_1,Butaro,None,None,Sector,Sector,4402,None,"MULTIPOLYGON (((29.82680 -1.42466, 29.82673 -1..."
2,RWA,Rwanda,RWA.1_1,Amajyaruguru,None,RWA.1.1_1,Burera,None,RWA.1.1.3_1,Cyanika,None,None,Sector,Sector,4403,None,"MULTIPOLYGON (((29.75238 -1.39963, 29.75236 -1..."
3,RWA,Rwanda,RWA.1_1,Amajyaruguru,None,RWA.1.1_1,Burera,None,RWA.1.1.4_1,Cyeru,None,None,Sector,Sector,4404,None,"MULTIPOLYGON (((29.82700 -1.54741, 29.82691 -1..."
4,RWA,Rwanda,RWA.1_1,Amajyaruguru,None,RWA.1.1_1,Burera,None,RWA.1.1.5_1,Gahunga,None,None,Sector,Sector,4405,None,"MULTIPOLYGON (((29.70308 -1.46780, 29.70290 -1..."


In [23]:
df2['geometry'] = df2.apply(lambda z: Point(z.longitude, z.latitude), axis=1)
PointsGeodataframe = gpd.GeoDataFrame(df2)
v1 = len(PointsGeodataframe)
print(v1)
PointsGeodataframe['valid'] = PointsGeodataframe.is_valid
display(PointsGeodataframe.loc[PointsGeodataframe['valid'] == False])
missing_location = PointsGeodataframe[PointsGeodataframe['valid'] == False]
PointsGeodataframe = PointsGeodataframe[PointsGeodataframe['valid'] == True]
print(PointsGeodataframe.columns)
v2 = len(PointsGeodataframe)
print(v2)

3240


/tmp/ipykernel_20207/706697605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['geometry'] = df2.apply(lambda z: Point(z.longitude, z.latitude), axis=1)


,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,master_timestamp,school_archive,electricity_availability,internet_availability,number_of_students,number_of_teachers,computers,connectivity_type,geometry,valid


Index(['uuid', 'country', 'source', 'giga_school_id', 'source_school_id',
       'school_name', 'education_level', 'latitude', 'longitude', 'hex8',
       'dq_score', 'source_timestamp', 'master_timestamp', 'school_archive',
       'electricity_availability', 'internet_availability',
       'number_of_students', 'number_of_teachers', 'computers',
       'connectivity_type', 'geometry', 'valid'],
      dtype='object')
3240


In [24]:
df_rw = gpd.sjoin(PointsGeodataframe, admin3_gdf, op='within') 
print(df_rw.columns)
display(df_rw)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/tmp/ipykernel_20207/2306189924.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df_rw = gpd.sjoin(PointsGeodataframe, admin3_gdf, op='within')


Index(['uuid', 'country', 'source', 'giga_school_id', 'source_school_id',
       'school_name', 'education_level', 'latitude', 'longitude', 'hex8',
       'dq_score', 'source_timestamp', 'master_timestamp', 'school_archive',
       'electricity_availability', 'internet_availability',
       'number_of_students', 'number_of_teachers', 'computers',
       'connectivity_type', 'geometry', 'valid', 'index_right', 'GID_0',
       'NAME_0', 'GID_1', 'NAME_1', 'NL_NAME_1', 'GID_2', 'NAME_2',
       'NL_NAME_2', 'GID_3', 'NAME_3', 'VARNAME_3', 'NL_NAME_3', 'TYPE_3',
       'ENGTYPE_3', 'CC_3', 'HASC_3'],
      dtype='object')


,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,NAME_2,NL_NAME_2,GID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3
0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,886ade34e7fffff,...,Karongi,None,RWA.4.1.7_1,Murambi,None,None,Sector,Sector,3106,None
389,7515e65f-dd53-41b0-b237-8ad341fc14a5,Rwanda,government,G_RWA_00998,310623.0,EP NYAKABUYE,Primary,-2.154750,29.532190,886ade34e5fffff,...,Karongi,None,RWA.4.1.7_1,Murambi,None,None,Sector,Sector,3106,None
391,cdd1f874-ebda-4154-9b28-fb8e3f8db3e5,Rwanda,government,G_RWA_01003,310624.0,EP KAVUMU,Primary,-2.179080,29.585360,886ade34b3fffff,...,Karongi,None,RWA.4.1.7_1,Murambi,None,None,Sector,Sector,3106,None
401,7613aaf2-d1e4-4113-bf2b-b5c8f22ec34c,Rwanda,government,G_RWA_01030,310622.0,EP RUSENGE,Primary,-2.189220,29.549810,886ade3593fffff,...,Karongi,None,RWA.4.1.7_1,Murambi,None,None,Sector,Sector,3106,None
452,5c2ca252-e92c-4d91-b8f6-d697d96e6e1b,Rwanda,government,G_RWA_01148,521313.0,EP NYAKABUYE,Primary,-2.154750,29.532194,886ade34e5fffff,...,Karongi,None,RWA.4.1.7_1,Murambi,None,None,Sector,Sector,3106,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,0b322784-e178-4860-9668-464fb6c54671,Rwanda,government,G_RWA_03358,440407.0,KIRAMBO,Primary,-1.497440,29.833690,886adc759bfffff,...,Burera,None,RWA.1.1.4_1,Cyeru,None,None,Sector,Sector,4404,None
3121,772fefcc-d616-41f9-bc44-8df0ef22eb83,Rwanda,government,G_RWA_03195,551005.0,NTARUKA,Primary,-2.153330,30.695170,886ad836e7fffff,...,Kirehe,None,RWA.3.4.10_1,Nasho,None,None,Sector,Sector,5510,None
3495,ab468640-ed23-479d-8533-13259b5ef2dd,Rwanda,government,G_RWA_03590,551006.0,EPNYABUBARE,Primary,-2.071030,30.719920,886ad9d951fffff,...,Kirehe,None,RWA.3.4.10_1,Nasho,None,None,Sector,Sector,5510,None
3517,7cc55944-a714-4e84-ae12-8789627c58d4,Rwanda,government,G_RWA_03613,551004.0,MUKOMA,Primary,-2.130555,30.754028,886ad9d9a1fffff,...,Kirehe,None,RWA.3.4.10_1,Nasho,None,None,Sector,Sector,5510,None


In [25]:
rt_rw = df_rw.drop(['index_right', 'GID_0', 'GID_1',
       'NL_NAME_1', 'GID_2', 'NL_NAME_2', 'GID_3',
        'TYPE_3', 'ENGTYPE_3', 'CC_3', 'HASC_3', 'VARNAME_3', 'NL_NAME_3'], axis = 1)
rt_rw.reset_index(inplace = True)
print(rt_rw.columns)

Index(['index', 'uuid', 'country', 'source', 'giga_school_id',
       'source_school_id', 'school_name', 'education_level', 'latitude',
       'longitude', 'hex8', 'dq_score', 'source_timestamp', 'master_timestamp',
       'school_archive', 'electricity_availability', 'internet_availability',
       'number_of_students', 'number_of_teachers', 'computers',
       'connectivity_type', 'geometry', 'valid', 'NAME_0', 'NAME_1', 'NAME_2',
       'NAME_3'],
      dtype='object')


In [26]:
rt_rw.head()

,index,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,...,number_of_students,number_of_teachers,computers,connectivity_type,geometry,valid,NAME_0,NAME_1,NAME_2,NAME_3
0,0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,...,28,1,260.0,3G,POINT (29.54797 -2.15939),True,Rwanda,Iburengerazuba,Karongi,Murambi
1,389,7515e65f-dd53-41b0-b237-8ad341fc14a5,Rwanda,government,G_RWA_00998,310623.0,EP NYAKABUYE,Primary,-2.154750,29.532190,...,532,20,156.0,None,POINT (29.53219 -2.15475),True,Rwanda,Iburengerazuba,Karongi,Murambi
2,391,cdd1f874-ebda-4154-9b28-fb8e3f8db3e5,Rwanda,government,G_RWA_01003,310624.0,EP KAVUMU,Primary,-2.179080,29.585360,...,532,20,120.0,None,POINT (29.58536 -2.17908),True,Rwanda,Iburengerazuba,Karongi,Murambi
3,401,7613aaf2-d1e4-4113-bf2b-b5c8f22ec34c,Rwanda,government,G_RWA_01030,310622.0,EP RUSENGE,Primary,-2.189220,29.549810,...,537,14,143.0,None,POINT (29.54981 -2.18922),True,Rwanda,Iburengerazuba,Karongi,Murambi
4,452,5c2ca252-e92c-4d91-b8f6-d697d96e6e1b,Rwanda,government,G_RWA_01148,521313.0,EP NYAKABUYE,Primary,-2.154750,29.532194,...,87,2,372.0,3G,POINT (29.53219 -2.15475),True,Rwanda,Iburengerazuba,Karongi,Murambi


In [27]:
rt_rw.shape

(3240, 27)

In [30]:
rw_RFI = rt_rw[rt_rw['internet_availability']=='NONE']

In [31]:
rw_RFI.shape

(1448, 27)

In [32]:
rw_RFI.to_csv('./giga/personal/iyke/rwanda_RFI.csv',encoding='utf-8')